In [1]:
import keras
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchvision.transforms import ToPILImage
from torch.autograd import Variable
import math
import numpy as np
import csv
import pandas as pd

Using TensorFlow backend.


In [2]:
# code to download data from CIFAR-10 
train_dataset= dsets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=False)

In [3]:
# extracting images of car from the entire dataset
cars=[]

for i in range(0,len(train_dataset.train_labels)):
    if train_dataset[i][1] == 1 :
        cars.append(train_dataset[i])


In [ ]:
#display each image - data labelling
to_img = ToPILImage()
data= to_img(cars[1500][0])
#data.show()

In [ ]:
# data is created by storing each car type in the csv file
f = open('Data_Project_DL.csv', 'a')
#f.write('Hatchback')
f.write('\n')
f.close()

In [4]:
data= pd.read_csv("Data_Project_DL.csv", header=None)
data= np.array(data)
sedanIndex = []
sedanImages = []
hatchIndex = []
hatchImages = []
suvIndex = []
suvImages = []
conIndex = []
conImages = []
commonLabels = []
commonImages = []

In [6]:
#######
countSedan = 0
countSUV = 0
countHatch = 0
countCon = 0
for i in range(0, len(data)):
    if data[i]== 'Sedan':
        data[i]= 1
        sedanIndex.append(i)
        #sedanImages.append(cars[i][0])
        if countSedan <= 154:
            # np.insert(commonLabels, data[i])
           # print(data[i])
            commonLabels.append(1)
            commonImages.append(cars[i][0])
            countSedan +=1
    elif data[i]== 'SUV':
        data[i]= 2
        suvIndex.append(i)
        #suvImages.append(cars[i][0])
        if countSUV <= 154:
            commonLabels.append(2)
            commonImages.append(cars[i][0])
            countSUV +=1
    elif data[i]== 'Hatchback':
        data[i]= 2
        hatchIndex.append(i)
        #hatchImages.append(cars[i][0])
        if countHatch <= 154:
            commonLabels.append(3)
            commonImages.append(cars[i][0])
            countHatch +=1
    elif data[i]== 'Convertible':
        data[i]= 2
        conIndex.append(i)
        #conImages.append(cars[i][0])
        if countCon <= 154:
            commonLabels.append(4)
            commonImages.append(cars[i][0])
            countCon +=1
#print("Sedan count:"+ str(len(sedanIndex)))
#print("suv count:"+ str(len(suvIndex)))
#print("hatchback count:"+ str(len(hatchIndex)))
#print("convertible count:"+ str(len(conIndex)))
#print("Total labels in commonlist:" + str(len(commonLabels)))
#print("total images in common:"+ str(len(commonImages)))

##

In [7]:
commonLabels= pd.DataFrame(commonLabels)
commonLabels= np.array(commonLabels)

In [8]:
# separating training, validation and test data from created dataset
X_train= cars[0:1200][0][0].numpy().reshape(1, 32, 32, 3)
X_val= cars[1201:1501][0][0].numpy().reshape(1, 32, 32, 3)
#X_test= commonImages[1202:1501][0][0]
Y_train= data[0:1200]
#Y_train= np.array(Y_train).reshape(1200,1)
#Y_train = keras.utils.to_categorical(np.ravel(Y_train))
Y_val= data[1201:1501]
#Y_val = keras.utils.to_categorical(np.ravel(Y_val))
#Y_val= np.array(Y_val).reshape(300,1)
# Y_test= data[1202:1501]
# Y_test = np.array(Y_test)

In [118]:
# for i in range(0, len(Y_train)):
#     if  Y_train[i]=='Sedan':
#         Y_train[i]=1
#     if  Y_train[i]=='SUV':
#         Y_train[i]=2
#     if  Y_train[i]=='Hatchback':
#         Y_train[i]=3
#     if  Y_train[i]=='Convertible':
#         Y_train[i]=4  

In [9]:
X_train_1= []
for i in range(0, 1200):
    X_train_1.append(np.array(cars[i][0].numpy().reshape(1, 32, 32, 3)))    

In [10]:
X_train_2  = np.array(X_train_1).reshape(1200,32,32,3)
print(X_train_2.shape)

(1200, 32, 32, 3)


In [11]:
X_val_1= []
for i in range(1201, 1501):
    X_val_1.append(np.array(cars[i][0].numpy().reshape(1,32, 32, 3)))

In [12]:
X_val_2= np.array(X_val_1).reshape(300,32,32,3)

In [ ]:
#X_train_1.shape[0]

In [ ]:
#X_train = X_train_1.reshape(X_train_1.shape[0], 3, 32, 32)

In [13]:
X_train_1= X_train.astype('float32')
X_val_1= X_val.astype('float32')
X_train_1 /= 255
X_val_1 /= 255

In [15]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GaussianNoise
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

In [16]:
#Initialize CNN


classifier= Sequential()

#First Convolutional Layer
classifier.add(Convolution2D(512, 3, 3 , input_shape= (32, 32, 3), activation='relu'))


#Max Pool
classifier.add(MaxPooling2D(pool_size= (2,2)))


classifier.add(Dropout(0.25))

# Second Convolutional Layer

classifier.add(Convolution2D(1024, 3, 3, activation= 'relu'))

#Max pool

classifier.add(MaxPooling2D(pool_size= (2,2)))


classifier.add(Dropout(0.15))
#Third Conv

classifier.add(Convolution2D(2048, 3, 3, activation= 'relu'))

#Third Maxpool
classifier.add(MaxPooling2D(pool_size= (2,2)))

classifier.add(Dropout(0.10))
#Flatten

classifier.add(Flatten())

#Hidden_layer

classifier.add(Dense(output_dim=128, activation = 'relu'))
classifier.add(GaussianNoise(0.25))
classifier.add(BatchNormalization())
# classifier.add(Dense(output_dim=128, activation = 'relu'))
# classifier.add(Dense(output_dim=128, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(GaussianNoise(0.25))
# classifier.add(BatchNormalization())
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(GaussianNoise(0.25))
# classifier.add(BatchNormalization())
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(GaussianNoise(0.25))
# classifier.add(BatchNormalization())
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# classifier.add(Dense(output_dim=256, activation = 'relu'))
# # classifier.add(GaussianNoise(0.25))

#Outer_layer

classifier.add(Dense(output_dim=1, activation = 'sigmoid'))

#Compile

from keras import optimizers
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

opt = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
classifier.compile(optimizer= opt , loss= 'binary_crossentropy', metrics= ['accuracy'])

#Image Augmentation

from keras.preprocessing.image import ImageDataGenerator

C:\Users\Vicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  import sys
C:\Users\Vicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, (3, 3), activation="relu")`
C:\Users\Vicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2048, (3, 3), activation="relu")`
C:\Users\Vicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
C:\Users\Vicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`


In [17]:
datagen = ImageDataGenerator(
        featurewise_center=False,                         
        samplewise_center=False,                         
        featurewise_std_normalization=False,                         
        samplewise_std_normalization=True,                  
        zca_whitening=False,                         
        rotation_range=0,                         
        width_shift_range=0.1,                         
        height_shift_range=0.1,                         
        horizontal_flip=True,                         
        vertical_flip=True,
        fill_mode= 'nearest')                         


datagen.fit(X_train_2)





In [ ]:
classifier.fit_generator(datagen.flow(X_train_2, Y_train),
                    steps_per_epoch=len(X_train_1) / 10, epochs=10) #validation_data=(X_val_2, Y_val),workers=4 )

In [177]:
predict_k= classifier.predict(X_val_2)

In [ ]:
def build_classifier(optimizer):
    
    from keras.models import Sequential
    from keras.layers import Convolution2D
    from keras.layers import MaxPooling2D
    from keras.layers import Flatten
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import GaussianNoise
    from sklearn.model_selection import GridSearchCV
    from keras.wrappers.scikit_learn import KerasClassifier
    

    classifier= Sequential()
    classifier.add(Convolution2D(512, 3, 3 , input_shape= (32, 32, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size= (2,2)))
    classifier.add(Dropout(0.25))
    classifier.add(Convolution2D(1024, 3, 3, activation= 'relu'))
    classifier.add(MaxPooling2D(pool_size= (2,2)))
    classifier.add(Dropout(0.15))
    classifier.add(Convolution2D(2048, 3, 3, activation= 'relu'))
    classifier.add(MaxPooling2D(pool_size= (2,2)))
    classifier.add(Dropout(0.10))
    classifier.add(Flatten())
    
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(GaussianNoise(0.25))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=256, activation = 'relu'))
    classifier.add(Dense(output_dim=1, activation = 'sigmoid'))
    classifier.compile(optimizer= opt , loss= 'hinge', metrics= ['accuracy'])
    return classifier

classifier= KerasClassifier(build_fn= build_classifier)

parameters= { 'batch_size' : [10, 16, 32],'epochs': [30, 100, 200],'optimizer': ['Adagrad', 'RMSprop', 'SGD', 'adam']}

grid_search= GridSearchCV(estimator= classifier, param_grid= parameters, scoring='accuracy', cv=100)

grid_search= grid_search.fit(X_train_1, Y_train)

best_parameters= grid_search.best_params_

best_accuracy= grid_search.best_score_


In [19]:
X_train_svm= X_train_2.reshape(1200, 3072)
X_val_svm= X_val_2.reshape(300, 3072)

In [141]:
from sklearn import svm

clf= svm.LinearSVC()
clf.fit(X_train_svm, Y_train)

C:\Users\Vicky\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [142]:
predict= clf.predict(X_val_svm)

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, predict)

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, predict)